In [79]:
# zdielanie -> ano/nie


# filter nehnutelnosti na datasete

# filter -> cena per user
# filter -> lokalila
# filter -> typ
# filter -> stav
request_body_test = {
    "content":{
        "id":"b41aa44d-a04b-4436-b61e-eea038b5685a",
        "country": "false",
        "city":"null",
        "cityParts":[
            "Sever",
            "Staré Mesto"
        ],
        "maxPrice":300,
        "roommatePreferences":{
            "id":"d6b5e2d7-8b67-45b4-8ec8-ee069c8aac44",
            "ageFrom":"33.0",
            "ageTo":"59.99999999999999",
            "nationality":"INTERNATIONAL",
            "gender":"MALE"
        },
        "education":{
            "kindergarten":False,
            "primarySchool":False,
            "highSchool":False
        },
        "freeTime":{
            "theater":False,
            "cinema":False,
            "sport":False
        },
        "transportation":{
            "publicTransport":False,
            "nonMotor":False,
            "motor":False
        }
    },
    "statusMessage":"SUCCESSFUL"
}

In [92]:
import pandas as pd
import geopandas
import os
dataset_path = "_data"

In [136]:
def calc_intersection(df_merge, df_for_compare, colummn,x_for_compare, y_for_compare, radius_distance):
    for idx, item in df_merge.iterrows():
        gdf = geopandas.GeoDataFrame(df_merge[idx:idx+1], geometry=geopandas.points_from_xy(df_merge[idx:idx+1]["latitude"], df_merge[idx:idx+1]["longitude"]), crs="EPSG:4326")
        gdf=gdf.to_crs("EPSG:3857") #convert projection web mercator or find best epsg code, where your data have.
        gdf['buffergeometry25miles'] = gdf['geometry'].buffer(radius_distance) # 25 mile = 40233.6 meter radius
        gdf.set_geometry('buffergeometry25miles',inplace=True)
        givengdf = geopandas.GeoDataFrame(df_for_compare, geometry=geopandas.points_from_xy(x_for_compare, y_for_compare, crs="EPSG:4326"))
        givengdf=givengdf.to_crs("EPSG:3857")
        intersect_df = geopandas.overlay(gdf, givengdf, how='intersection', keep_geom_type=False)
        # print(len(intersect_df))
        df_merge.loc[idx, colummn] = len(intersect_df)

In [126]:
import requests

r = requests.get(url = 'http://192.168.233.82:8080/rent/all')
j = r.json()
j

# # df = pd.DataFrame([[d['v'] for d in x['c']] for x in j['rows']],
# #                   columns=[d['label'] for d in j['cols']])
#
df_merge = pd.DataFrame.from_dict(j["content"])
df_merge

df_schools = pd.read_csv("schools.csv")

filename = "open_ke_mhd.geojson"
file = open(os.path.join(dataset_path,filename))
df_mhd = geopandas.read_file(file)
df_mhd.drop_duplicates(subset='zastavka_nazov', inplace=True)

dataset_path = "_data"
print("DATASET LOADING -> Schools")
filename = "culture.csv"
df_culture  = pd.read_csv(filename)

filename = "cycle_points.geojson"
file = open(os.path.join(dataset_path,filename))
df_cycle_points = geopandas.read_file(file)
df_cycle_points["y"] = df_cycle_points.apply(lambda row: float(row.geometry[0].y), axis=1)
df_cycle_points["x"] = df_cycle_points.apply(lambda row: float(row.geometry[0].x), axis=1)

DATASET LOADING -> Schools


/var/folders/mt/zbsgqls91d5c1dt725ckq4lc0000gn/T/ipykernel_56349/290371605.py:28: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  df_cycle_points["y"] = df_cycle_points.apply(lambda row: float(row.geometry[0].y), axis=1)
/var/folders/mt/zbsgqls91d5c1dt725ckq4lc0000gn/T/ipykernel_56349/290371605.py:29: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  df_cycle_points["x"] = df_cycle_points.apply(lambda row: float(row.geometry[0].x), axis=1)


In [131]:
df_merge

,id,name,description,rentType,country,city,cityPart,street,houseNumber,latitude,...,numberOfRooms,numberOfOccupants,photo,school,culture,green,health,edu_kindergarten,edu_primarySchool,edu_highSchool
0,21a9172b-da6a-465b-9b25-cb77656533c3,Prenajom 3i byt secovce,"Prenajmem 3 izbovy byt v Secovciach, Obchodna ...",Rodinný dom,None,Košice,Vyšné Opátske,Včelárska paseka,1,21.286188,...,3,4,https://s3.spacecode.sk/spliti/dtit_hack/3cccf...,0.0,0.0,0.965,0.000,0,0,0
1,d43b9242-74eb-40fa-beca-8d65e480f10c,"Exkluzívny loft s terasou na prenájom, Smižany...",Realitná kancelária Living Reality pripravuje ...,Rodinný dom,None,Košice,Západ,Topoľčianska,30,21.224554,...,1,11,https://s3.spacecode.sk/spliti/dtit_hack/350b6...,0.5,0.0,0.275,1.635,0,0,0
2,e770890a-c511-46ab-b055-f6397addaf42,"5-izbový mezonet v centre s dvojgarážou, pivnicou",Ponúkame do nájmu nadštandardný 5-izbový mezon...,Rodinný dom,None,Košice,Krásna,Keldišova,19,21.306923,...,5,12,https://s3.spacecode.sk/spliti/dtit_hack/79319...,0.0,0.0,0.965,0.000,0,0,0
3,c0bc4e26-bb48-495c-a211-e76103696a27,Na prenájom veľký 2i byt v RD Košice Sever -10...,"Ponúkame na prenájom veľký 2 izbový , rekonštr...",Bytový dom,None,Košice,Šaca,Železiarenská,40,21.172905,...,2,7,https://s3.spacecode.sk/spliti/dtit_hack/08b5c...,1.0,0.0,1.600,1.175,0,0,0
4,8ec1f4d3-10ff-4a2f-808a-a9d3ee78c314,"Prenajmem 3 izbový byt, Košice",Prenajmeme zrekonštruovaný a zariadený 3 izbov...,Bytový dom,None,Košice,Sever,Vihorlatská,8,21.239626,...,3,5,https://s3.spacecode.sk/spliti/dtit_hack/81a68...,1.0,0.0,2.620,1.135,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,18fdba84-b8b9-49fc-a596-234175106055,Byt v novostavbe s klimatizáciou a parkovacím ...,"Na prenájom moderný 2izbový byt v novostavbe ""...",Bytový dom,None,Košice,Nad jazerom,Čechovova,29,21.305096,...,2,3,https://s3.spacecode.sk/spliti/dtit_hack/bfc8f...,1.5,0.0,1.765,0.135,0,0,0
101,3c708014-ac90-48d1-96e6-fe4995026bc7,3 izbový byt na prenájom v tesnej blízkosti ce...,Realitná kancelária TOBI Real Košice Vám ponúk...,Bytový dom,None,Košice,Sever,Palkovičova,8,21.252386,...,3,4,https://s3.spacecode.sk/spliti/dtit_hack/48c9d...,3.0,1.5,2.175,2.445,0,0,0
102,ce4843f9-66d4-4ab8-8099-149b8d689c52,2 izbový byt v rezidencii PRI RADNICI Strojáre...,Ponúkame na prenájom kompletne zariadený byt 2...,Bytový dom,None,Košice,Západ,Strážovská,2,21.239662,...,2,3,https://s3.spacecode.sk/spliti/dtit_hack/151be...,4.0,1.5,2.475,5.000,0,0,0
103,0bc3ca02-5348-4592-a300-6adb1b32342f,Na prenájom samostatná nepriechodná izba 20 m2,Na prenájom samostatná nepriechodná izba 20 m2...,Bytový dom,None,Košice,Západ,Kežmarská,7,21.230827,...,1,1,https://s3.spacecode.sk/spliti/dtit_hack/5a279...,3.5,0.0,2.065,2.945,0,0,0


In [133]:
# FILTER BY PRICE
df_merge = df_merge.loc[df_merge["pricePerPerson"] <= request_body_test["content"]["maxPrice"]]

# FILTER BY LOCALITY
df_merge = df_merge.loc[df_merge["cityPart"].isin(request_body_test["content"]["cityParts"])]
df_merge.reset_index(inplace=True)

In [134]:
# FILTER BY SCHOOL
# "education":{
#     "kindergarten":False,
#     "primarySchool":False,
#     "highSchool":False
# },
request_body_test["content"]["education"]["kindergarten"]

False

In [137]:
df_merge["edu_kindergarten"] = 0
df_merge["edu_primarySchool"] = 0
df_merge["edu_highSchool"] = 0
column_for_compare = []
sort_order = []
# if request_body_test["content"]["education"]["kindergarten"]:
if True:
    column_for_compare.append("edu_kindergarten")
    sort_order.append(False)
    df_tmp = df_schools.loc[df_schools["type"]=="elementary"]
    print(df_tmp.shape)
    calc_intersection(df_merge=df_merge,
                      df_for_compare=df_tmp,
                      x_for_compare=df_tmp["x"],
                      y_for_compare=df_tmp["y"],
                      colummn="edu_kindergarten",
                      radius_distance=750)

# if request_body_test["content"]["education"]["primarySchool"]:
if True:
    column_for_compare.append("edu_primarySchool")
    sort_order.append(False)
    df_tmp = df_schools.loc[df_schools["type"]=="primary"]
    print(df_tmp.shape)
    calc_intersection(df_merge=df_merge,
                      df_for_compare=df_tmp,
                      x_for_compare=df_tmp["x"],
                      y_for_compare=df_tmp["y"],
                      colummn="edu_primarySchool",
                      radius_distance=750)

# if request_body_test["content"]["education"]["highSchool"]:
if True:
    column_for_compare.append("edu_highSchool")
    sort_order.append(False)
    df_tmp = df_schools.loc[df_schools["type"]=="high"]
    print(df_tmp.shape)
    calc_intersection(df_merge=df_merge,
                      df_for_compare=df_tmp,
                      x_for_compare=df_tmp["x"],
                      y_for_compare=df_tmp["y"],
                      colummn="edu_highSchool",
                      radius_distance=750)


(100, 29)
(62, 29)
(47, 29)


In [138]:
# "transportation":{
#     "publicTransport":False,
#     "nonMotor":False,
#     "motor":False
# }
df_merge["trans_nonMotor"] = 0
df_merge["trans_public"] = 0

# if request_body_test["content"]["transportation"]["nonMotor"]:
if True:
    column_for_compare.append("trans_nonMotor")
    sort_order.append(False)
    calc_intersection(df_merge=df_merge,
                      df_for_compare=df_cycle_points,
                      x_for_compare=df_cycle_points["x"],
                      y_for_compare=df_cycle_points["y"],
                      colummn="trans_nonMotor",
                      radius_distance=1000)

# if request_body_test["content"]["transportation"]["publicTransport"]:
if True:
    column_for_compare.append("trans_public")
    sort_order.append(False)
    print(df_tmp.shape)
    calc_intersection(df_merge=df_merge,
                      df_for_compare=df_mhd,
                      x_for_compare=df_mhd["x"],
                      y_for_compare=df_mhd["y"],
                      colummn="trans_public",
                      radius_distance=500)

(47, 29)


In [139]:
df_out = df_merge.sort_values(column_for_compare, ascending = sort_order)

In [141]:
df_out

,index,id,name,description,rentType,country,city,cityPart,street,houseNumber,...,photo,school,culture,green,health,edu_kindergarten,edu_primarySchool,edu_highSchool,trans_nonMotor,trans_public
10,49,b12235d3-3070-4adc-83ef-e3dc4bf16269,"Veľkometrážny 2 izb. byt 86 m2 v centre, ul. K...",Peter Zakuťanský z realitnej kancelárie AURUM ...,Bytový dom,None,Košice,Staré Mesto,Zimná,20,...,https://s3.spacecode.sk/spliti/dtit_hack/57549...,4.5,3.5,5.000,4.890,5,3,1,49,3
3,17,1493961e-5353-4a73-9ceb-bbfd50812b96,Prenájom priestranného a slnečného 4 izbového ...,Ponúkam na dlhodobý prenájom zariadený štvoriz...,Bytový dom,None,Košice,Staré Mesto,Námestie L. Novomeského,1,...,https://s3.spacecode.sk/spliti/dtit_hack/3d183...,4.5,5.0,4.100,5.000,5,1,3,60,1
4,19,d599460f-669d-4fb9-a156-f2ce04174b71,"Prenájom 3 izbového bytu, Alvinczyho ul. , Koš...",049-213-BABAa\n\nBarbora Bartková a BeMi reali...,Rodinný dom,None,Košice,Sever,Družstevná,3,...,https://s3.spacecode.sk/spliti/dtit_hack/5b25e...,4.5,3.0,2.920,2.715,5,1,3,24,2
18,85,141d95f7-4277-4f6f-88fc-1442a01c5ef2,"Ponúkame na prenájom 2-izbový byt s terasou, 6...","Ponúkaný byt sa nachádza na Terase, ul. Idansk...",Bytový dom,None,Košice,Staré Mesto,Haviarska,8A,...,https://s3.spacecode.sk/spliti/dtit_hack/89131...,4.5,2.5,3.500,2.715,4,2,3,20,3
12,61,77781462-a7be-4e88-ba88-782588b0d6ef,"2 izb. Nad Jazerom, Bukovecká ul., OV, 3p, čia...",REZERVOVANÝ\n\n\nPONUKY NA NÁJOM PROSÍM ZASIEL...,Bytový dom,None,Košice,Sever,Kustrova,1,...,https://s3.spacecode.sk/spliti/dtit_hack/7425c...,3.0,0.5,1.680,2.445,4,2,0,16,1
9,34,1746c0d8-a8ab-485c-8f04-a0f8cef369a9,"Na prenájom 3 izb. byt, 68m2, loggia, ul. Sofi...",Exkluzívne Vám ponúkame na dlhodobý prenájom 3...,Rodinný dom,None,Košice,Sever,Cimborkova,2,...,https://s3.spacecode.sk/spliti/dtit_hack/dfff0...,4.0,2.0,1.900,3.120,4,1,3,20,1
23,101,3c708014-ac90-48d1-96e6-fe4995026bc7,3 izbový byt na prenájom v tesnej blízkosti ce...,Realitná kancelária TOBI Real Košice Vám ponúk...,Bytový dom,None,Košice,Sever,Palkovičova,8,...,https://s3.spacecode.sk/spliti/dtit_hack/48c9d...,3.0,1.5,2.175,2.445,4,1,1,25,1
6,27,da2a1123-f4ab-4fdf-8ab3-363871b5276a,Ponúkame Vám na dlhodobý prenájom 2-izbový byt...,Ponúkame Vám na dlhodobý prenájom 2-izbový byt...,Rodinný dom,None,Košice,Staré Mesto,Bellova,2,...,https://s3.spacecode.sk/spliti/dtit_hack/e6eaa...,3.0,2.0,2.950,1.770,4,1,1,10,3
16,77,9ab07330-a928-4176-b599-825f6cb7758e,"2-izbový, prenájom, 50 m2, rýchla dostupnosť d...",Ponúkame Vám na prenájom pekný 2-izbový byt po...,Rodinný dom,None,Košice,Sever,Slovenského,17,...,https://s3.spacecode.sk/spliti/dtit_hack/88151...,2.5,2.0,1.570,2.040,4,1,0,14,1
14,72,bd4e37ad-99b2-4bd9-90dd-bfa94a9d14a9,"Tehlový 3 izb. byt 66m2 + balkón, prenájom, Ha...",Vladimír Gaži a Partnersreal Vám ponúka exkluz...,Rodinný dom,None,Košice,Sever,Mánesova,10,...,https://s3.spacecode.sk/spliti/dtit_hack/bed3b...,2.0,0.5,1.350,1.770,4,0,0,10,1


In [142]:
df_out['id']

10    b12235d3-3070-4adc-83ef-e3dc4bf16269
3     1493961e-5353-4a73-9ceb-bbfd50812b96
4     d599460f-669d-4fb9-a156-f2ce04174b71
18    141d95f7-4277-4f6f-88fc-1442a01c5ef2
12    77781462-a7be-4e88-ba88-782588b0d6ef
9     1746c0d8-a8ab-485c-8f04-a0f8cef369a9
23    3c708014-ac90-48d1-96e6-fe4995026bc7
6     da2a1123-f4ab-4fdf-8ab3-363871b5276a
16    9ab07330-a928-4176-b599-825f6cb7758e
14    bd4e37ad-99b2-4bd9-90dd-bfa94a9d14a9
22    51f5e212-3745-4cab-9a77-e4d4ea693356
7     b5cf7971-b2af-435c-92c0-28c4158f3ef3
1     e6d03e74-c3c7-40f0-bb1f-23d265a52325
21    c248912e-36be-4175-bf8d-bcd54566c1d0
8     8c9ee459-6226-40a8-b9d1-58a3542c0895
0     8ec1f4d3-10ff-4a2f-808a-a9d3ee78c314
15    ccf445e9-85d6-4dfa-b1e0-2e8c19ac74c2
2     0220edeb-d98a-4872-a58c-3a8470b1b0ff
5     1358abec-50fc-4f3a-b423-92df9c878445
20    26fc6c1f-742c-4c78-8245-b6046383a54a
11    87a260bb-d1df-430f-9e8b-258232cfc04e
19    9387a1f9-898f-46d8-ab93-ea35dea44d1b
17    dbb69470-74b7-4958-81e8-582a410b0d4f
13    e922b

In [151]:
json_list = []

for id in df_out['id']:
    for obj in j['content']:
        if obj["id"] == id:
           json_list.append(obj)

In [152]:
json_list

[{'id': 'b12235d3-3070-4adc-83ef-e3dc4bf16269',
  'name': 'Veľkometrážny 2 izb. byt 86 m2 v centre, ul. Kováčska',
  'description': 'Peter Zakuťanský z realitnej kancelárie AURUM vám ponúka na prenájom tento priestranný 2-izbový byt o rozlohe 86 m2 v centre Košíc, na ul. Kováčska.\n\nAk ste radi v centre diania a chcete mať všetko na dosah, táto ponuka je určená práve Vám.\nZÁKLADNÉ ÚDAJE:\nDruh nehnuteľnosti: 2-izbový byt\nCelková plocha: 86 m2\nVlastníctvo: osobné\nStav nehnuteľnosti: Kompletná staršia rekonštrukcia\nPoschodie: 2/3\nVýťah: NIE\nOrientácia: V/Z\n\nDISPOZÍCIA: \nvstupná chodba, kúpeľňa s WC, kuchyňa, 2x veľká spálňa\n\nPOPIS BYTU:\n• Byt sa nachádza na ideálnom 2. poschodí z 3 bez výťahu\n• v prípade potreby z jednej spálne je možné spraviť obývaciu izbu\n• Voľný ihneď\n• Izby sú priestranné\n\nVÝHODY BYTU:\n• Pavlač orientovaná na západ\n• Orientácia V/Z - slnečný byt\n• Vynikajúca občianska vybavenosť\n\nLOKALITA A OKOLIE:\n• Byt sa nachádza v samom centre Košíc, na 

In [ ]:
response = {
    "responseCode" : "SUCCESSFUL",
    "content" : json_list
}